In [1]:
## Setup 
import requests 
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200') 


### Create Index
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}
index_name = "course-questions"
if es_client.indices.exists(index=index_name):
    es_client.indices.delete(index=index_name)
es_client.indices.create(index=index_name, body=index_settings)


### Read the document from the url given and index populate course-questions index
docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

from tqdm.auto import tqdm
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [2]:
## Common functions
def elastic_search(query, size=3, filter_term=None):
    search_query = {
        "size": size,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                }
            }
        }
    }

    # Add filter term if provided
    if filter_term:
        search_query["query"]["bool"]["filter"] = {"term": filter_term}

    response = es_client.search(index=index_name, body=search_query)
    return response['hits']['hits']

In [3]:
#Q3
elastic_search("How do execute a command on a Kubernetes pod?", 5)

[{'_index': 'course-questions',
  '_id': 'sM3CdJcBZVM7YVmco5py',
  '_score': 44.135918,
  '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
   'section': '5. Deploying Machine Learning Models',
   'question': 'How do I debug a docker container?',
   'course': 'machine-learning-zoomcamp'}},
 {'_index': 'course-questions',
  '_id': 'P83CdJcBZVM7YVmcpJuQ',
  '_score': 35.30038,
  '_source': {'text': 'Deploy and Access the Kubernetes Dashboard\nLuke',
   'section': '10. Kubernetes and TensorFlow Serving',
   'question': 'Kubernetes-dashboard',
   'course': 'machine-learning-zoomcamp'}},
 {'_index': 'course-questions',
  '_id': '0M3CdJcBZVM7YVmco5qv',
  '_score': 33.430336,
  '_source': {'text': 'You

In [4]:
elastic_search("How do copy a file to a Docker container?", 3, filter_term={"course": "machine-learning-zoomcamp"})

[{'_index': 'course-questions',
  '_id': 'sM3CdJcBZVM7YVmco5py',
  '_score': 72.965546,
  '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
   'section': '5. Deploying Machine Learning Models',
   'question': 'How do I debug a docker container?',
   'course': 'machine-learning-zoomcamp'}},
 {'_index': 'course-questions',
  '_id': 'z83CdJcBZVM7YVmco5qs',
  '_score': 66.40804,
  '_source': {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_

In [5]:
#Q5
### The elastic search here has to return documents, not just hits. So we will just define a new function here rather than re-using the one we have already created above
def elastic_search(query):
    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

    context_template = """
Q: {question}
A: {text}
""".strip()

    context = "\n\n".join([
        context_template.format(
            question=doc['question'],
            text=doc['text']
        ) for doc in search_results
    ])

    prompt = prompt_template.format(question=query, context=context)
    return prompt

query = "How do copy a file to a Docker container?";
search_results = elastic_search(query)
prompt = build_prompt(query, search_results)
len(prompt)

1446

In [6]:
# Q6
import tiktoken

# Load tokenizer
encoding = tiktoken.encoding_for_model("gpt-4o")

# Count tokens
token_count = len(encoding.encode(prompt))
print("Token count:", token_count)

Token count: 320
